In [11]:
import numpy as np                  # type: ignore
import napari                       # type: ignore     
import myfunctions as mf            # type: ignore

##### Reading 4D sequence

In [2]:
sequence = mf.read_4Dsequence('P28A_FT_H_Exp1', first_slice=100, last_slice=104+1)

threshold = mf.find_threshold(sequence, target=5700)

segmented_sequence = mf.segment4D(sequence, threshold, filtering3D=True, filtering4D=True, smallest_3Dvolume=10)

Sequence collected in 0.19 s

Finding threshold...
Threshold=1.94 found in 0.36 s



Backward propagation: 100%|██████████| 53/53 [00:03<00:00, 16.44it/s]


Filtering...


Inconsistent agglomerates removal: 100%|██████████| 44/44 [00:07<00:00,  5.92it/s]

Filtering completed in 8.05 s



In [12]:
viewer = napari.Viewer()

images = [viewer.add_image(sequence, name='Volume', opacity=0.4)]

labels = [viewer.add_labels(segmented_sequence, name='Labels', blending='additive', opacity=0.8)]

settings = napari.settings.get_settings()
settings.application.playback_fps = 5
viewer.dims.current_step = (0, 0)

##### Reading 3D sequence

In [13]:
sequence = mf.read_3Dsequence('P28A_FT_H_Exp1', time=150)

threshold = mf.find_threshold(sequence)

segmented_sequence = mf.segment3D(sequence, threshold)

Finding threshold...
Threshold=2.00 found in 0.19 s



In [17]:
viewer = napari.Viewer()
_ = viewer.add_image(sequence, opacity=0.4)
_ = viewer.add_labels(segmented_sequence, blending='additive', opacity=0.8)
settings = napari.settings.get_settings()
settings.application.playback_fps = 10
viewer.dims.current_step = (0, 0)